In [109]:
###Importamos las librerias necesarias:
##Importamos Pandas para trabajar con DataFrames
import pandas as pd

##Importamos Numpy para incluir mas funciones matematicas
import numpy as np

##Importamos MatPlotLib para realizar representaciones
import matplotlib.pyplot as plt
from matplotlib import gridspec #Es un paquete de matplotlib que permite realizar figuras de varias subfiguras

##Importamos la funcion display de la libreria IPython.display, que permite mostrar contenido de manera mas enriquecida
from IPython.display import display

##Importamos el modulo time para poder medir el tiempo de ejecución del codigo
import time

In [110]:
##Definimos la funcion para leer el archivo completo
def read_MBS_data_full(path):
    ##Cargamos los datos en un data frame
    inputfile = pd.read_csv(path, sep = "\t", header = None, dtype = np.float64, chunksize = None) #Tabulaciones como separador, sin encabezado y sin realizar division del dataframe
    df = inputfile.drop([1, 2, 3, 4, 129, 130], axis = 1, errors = 'ignore') #Elimino las columnas asociadas a las coordenadas y las dos ultimas que son 0 y Nan

    #Cambio las etiquetas de las columnas para que coincidan con el tiempo y el canal correspondiente
    etiquetas = ["tiempo"] + list(range(1, 125))
    df.columns = etiquetas

    return df #Devuelve un data frame
    

##Definimos la funcion para leer el archivo en chunks
def read_MBS_data_in_chunks(path, chunksize = 10000):
    ##Generamos el objeto iterable de los datos
    inputfile = pd.read_csv(path, sep = "\t", header = None, dtype = np.float64, chunksize = int(chunksize)) #Tabulaciones como separador, sin encabezado, con division del dataframe
    #Defino las etiquetas
    etiquetas = ["tiempo"] + list(range(1, 125))
    #Itero sobre el objeto iterable generado para realizr modeificaciones a cada chunk
    for chunk in inputfile:
        chunk.drop([1, 2, 3, 4, 129, 130], axis = 1, inplace = True, errors = 'ignore') #Elimino las columnas seleccionadas, reemplalzando el archivo en lugar de generar uno nuevo
        chunk.columns = etiquetas #Cambio las etiquetas del dataframe en cada chunk

        #Al emplear la sentencia yield estamos convirtiendo la funcion en una funcion generadora, proporcionando un objeto iterable como resultado
        yield chunk #Sustituye al return


In [114]:
##Funcion para detectar la presencia de eventos en cada observacion
def detect_events(row):
    #Implementacion del algoritmo de deteccion de eventos para cada fila
    #Inicializamos los marcadores
    flag1 = 0
    flag2 = 0
    flag3 = 0
    time = row["tiempo"]
    col_counter = 1

    #Empezamos a iterar sobre los elementos de la fila 
    for column in row[1:]: 
        if (column < 10000) and (flag1 == 0): #Comprobamos si se produce una caida en la señal. Si se cumple se marca flag1
            flag1 = 1
            time_coincidence = col_counter
        
        if (column > 20000) and (flag1 == 1): #Comprobamos si se produce una subida en la señal despues de haberse producido una caida, lo que indica evento. Si se cumple se marca flag2
            flag2 = 1
        
        if (column < 15000) and (flag2 == 1): #Si habiendose producido una coincidencia (flag2 = 1) se produce otra, se marca para descartar el evento (flag3 = 1) y se resetean los marcadores
            flag3 = 1
            flag2 = 0
            flag1 = 0
        else: 
            col_counter += 1
            continue

    #Se comprueban los marcadores para saber si se ha dado una coincidencia aislada
    #Se devvuelve si se produce coincidencia y el tiempo en el que se produce
    if (time != 0) and (flag1 == 1) and (flag2 == 1) and (flag3 != 1): #Si se ha detectado solo un evento aislado se devuelve 1 (Se comprueba tambien que el tiempo no sea 0, lo cual no seria valido)
        return {'Coincidencia': 1, 'Time_event': time + time_coincidence - 1}
    else:   #Si no se ha detectado evento o si se ha producido una coincidencia multiple se devuelve 0 
        return {'Coincidencia': 0, 'Time_event': 0}

In [112]:
###Función para el proceso de obtencion de las coincidencias en caso de cargar el archivo de datos original al completo
def data_procesing_full(path):
    #Leemos el DataFrame con una de las funciones definidas (cargamos el archivo al completo)
    df = read_MBS_data_full(path)

    ##Detectamos las coincidencias
    df[["Coincidencia", "Time_event"]] = df.apply(detect_events, axis = 1, result_type='expand') #La funcion devuelve si hay coincidencia y el tiempo en el que se produce (como devuelve un diccionario con dos keys se emplea el argumento result_type='expand', para descomprimir el resultado)
    df["Num_event"] = df.index + 1 #Definimos el número de evento. Más 1 para que el número de evento empiece en 1 y no en 0
    print(df.value_counts("Coincidencia")) #Recuento de coincidencias
    #Guardo el DataFrame que emplearé para entrenar la Neural Network (NN)
    df.to_csv("./Data/NN_coincidencias_data.dat", index = True, sep = "\t")

    ##Genero un nuevo DataFrame filtrando las observaciones que producen coincidencias
    df_coincidencias = df[df["Coincidencia"] == 1]
    df_coincidencias.index = range(1, len(df_coincidencias) + 1) #Cambio el indice para que empiece en 1

    #En un nuevo dataframe guardo el índice, el número de coincidencia y el tiempo de la coincidencia
    final_df = df_coincidencias[["Num_event", "Time_event"]].copy()
    #Genero el archivo de texto con los resultados del muestreo
    final_df.to_csv("./Data/Coincidencias_MBS.dat", index = True, sep = "\t")


###Función para el proceso de obtencion de las coincidencias en caso de cargar el archivo de datos original por partes para no sobrecargar la memoria
def data_procesing_in_chunks(path):
    #Leemos el archivo de datos por partes 
    df_in_chunks = read_MBS_data_in_chunks(path, chunksize = 100)

    #Creo los archivos sobre los que se van a escribir los resultados, definiendo unicamente el encabezado
    df_NN_vacio = pd.DataFrame(columns = ["tiempo"] + list(range(1,125)) + ["Coincidencia", "Time_event", "Num_event"]) #Encabezado del DataFrame para la NN
    df_coincidencias_vacio = pd.DataFrame(columns = ["Num_event", "Time_event"]) #Encabezado del archivo de coincidencias
    df_NN_vacio.to_csv("./Data/NN_coincidencias_data.dat", index = True, sep = "\t", header = True, mode = "w") #Creacion del archivo de NN
    df_coincidencias_vacio.to_csv("./Data/Coincidencias_MBS.dat", index = True, sep = "\t", header = True, mode = "w") #Creacion del archivo de coincidencias


    ##Iteramos sobre cada parte del archivo para trabajar con los diferentes segmentos
    count = 0 #Variable para llevar el recuento de las coincidencias en cada chunk

    for chunk in df_in_chunks:
        #Detectamos las coincidencias
        chunk[["Coincidencia", "Time_event"]] = chunk.apply(detect_events, axis = 1, result_type='expand')
        chunk["Num_event"] = chunk.index + 1 #Definimos el número de evento. Más 1 para que el número de evento empiece en 1 y no en 0
        #Vamos guardando la parte correspondiente a cada chunk en el mismo archivo que se usara para entrenar la NN
        chunk.to_csv("./Data/NN_coincidencias_data.dat", index = True, sep = "\t", header = False, mode = "a")

        ##Genero un nuevo DataFrame filtrando las observaciones que producen coincidencias
        chunk_coincidencias = chunk[chunk["Coincidencia"] == 1]
        chunk_coincidencias.index = range(1 + count, count + len(chunk_coincidencias) + 1) #Cambio el indice para que empiece en 1
        count = count + len(chunk_coincidencias)

        #En un nuevo dataframe guardo el índice, el número de coincidencia y el tiempo de la coincidencia
        final_chunk = chunk_coincidencias[["Num_event", "Time_event"]].copy()
        #Genero el archivo de texto con los resultados del muestreo
        final_chunk.to_csv("./Data/Coincidencias_MBS.dat", index = True, sep = "\t", header = False, mode = "a")

    print("El número de coincidencias totales detectadas es: {}".format(count))

In [113]:
##Se pregunta si la caga de datos se quiere hacer de manera segmentada o a la vez
decision = input("Introduce si quieres leer el archivo de datos cargandolo a la vez o de manera segmentada: True (carga segmentada), False (carga a la vez)")

##Inicio del contador de tiempos
start = time.time()

##Ruta al archivo de datos
MBS_data = "./Data/MBS_data.dat"

##Ejecutamos el bloque de comandos asociado a la carga simultanea o segmentada dependiendo de la decision introducida
if decision == "False":
    data_procesing_full(MBS_data)

if decision == "True":
    data_procesing_in_chunks(MBS_data)

##Fin del contador de tiempos
end = time.time()
#Tiempos transcurrido
execution_time = end - start
print("El tiempo de ejecucion del codigo ha sido: {} segundos".format(execution_time))

El número de coincidencias totales detectadas es: 138
El tiempo de ejecucion del codigo ha sido: 0.1058359146118164 segundos
